# Customers SCD2 Transformation - Silver Layer

## Summary
- Purpose: Build SCD Type 2 (slowly changing dimension) table for customers with historical tracking.
- Inputs: `capstone.bronze.customers` Delta table
- Outputs: `capstone.silver.customers_scd2` Delta table (SCD Type 2 history)
- Audit: Calls `audit_log(spark, table_name, log_path)` after major merge/write operations.

## Key Transformations
- Compute hash values for dedup/equals
- Merge staged data into SCD2 table using `MERGE` logic
- Preserve historical records and mark `is_current` flags

## Usage
- Run after Bronze ingestion; ensure `customers_scd2` exists before merge operations.


In [ ]:
dbutils.widgets.text("catalog", "capstone", "Enter the Catalog: ")

In [ ]:
from pyspark.sql.functions import col, lit, current_timestamp, monotonically_increasing_id, when
from capstone_pipeline.main import transform_hashvalue, audit_log


table_name = f'{dbutils.widgets.get("catalog")}.silver.customers_scd2'
log_path = f'/Volumes/{dbutils.widgets.get("catalog")}/meta/history'

In [ ]:
spark.sql(f"""
          CREATE TABLE IF NOT EXISTS {dbutils.widgets.get("catalog")}.silver.customers_scd2 (
            customer_key BIGINT,
            customer_id STRING,
            name STRING,
            email STRING,
            region STRING,
            hash_value STRING,
            start_date TIMESTAMP,
            end_date TIMESTAMP,
            is_current BOOLEAN,
            _ingest_timestamp TIMESTAMP,
            _source_file_name STRING
            ) USING DELTA;
          """)

In [ ]:

dfcustomers = spark.table('{dbutils.widgets.get("catalog")}.bronze.customers')
dfcustomers_staged = (dfcustomers
            .drop("_rescue")
            .transform(transform_hashvalue, ["name", "contact.email", "region"])
            .withColumn("email", col("contact.email"))
            .withColumn("start_date", current_timestamp())
            .withColumn("end_date", lit(None).cast("timestamp"))
            .withColumn("is_current", lit(True)))

dfcustomers_staged.createOrReplaceTempView("dfcustomers_staged")

display(spark.sql(f"SELECT * FROM dfcustomers_staged LIMIT 10"))


In [ ]:
dfcustscd = spark.table(f'{dbutils.widgets.get("catalog")}.silver.customers_scd2')
dfcustscdmaxkey = dfcustscd.agg({"customer_key": "max"})

In [ ]:
dfinsert = (dfcustomers_staged.alias("staged")
    .join(dfcustscd.filter(col("is_current") == True).alias("src"), on=["hash_value","customer_id"], how="inner")
    .select(
        "src.customer_key",
        "staged.customer_id",
        "staged.name",
        "staged.email",
        "staged.region",
        "staged.hash_value",
        "staged.start_date",
        "staged._ingest_timestamp",
        "staged._source_file_name"
    )
    # .withColumn("customer_key", lit(dfcustscdmaxkey.collect()[0][0]) + monotonically_increasing_id())
    .withColumn("end_date", lit(None).cast("timestamp"))
    .withColumn("is_current", lit(True)))

dfinsert.createOrReplaceTempView("dfinsert")

In [ ]:
dfinsert1 = (dfcustomers_staged.alias("staged")
    .join(dfcustscd.filter(col("is_current") == True).alias("src"), on=["hash_value","customer_id"], how="left")
    .select(
        "staged.customer_id",
        "staged.name",
        "staged.email",
        "staged.region",
        "staged.hash_value",
        "staged.start_date",
        "staged._ingest_timestamp",
        "staged._source_file_name"
    )
    .withColumn("customer_key", lit(dfcustscdmaxkey.collect()[0][0]) + monotonically_increasing_id())
    .withColumn("end_date", lit(None).cast("timestamp"))
    .withColumn("is_current", lit(True)))

dfinsert1.createOrReplaceTempView("dfinsert1")

In [ ]:
%skip 
merge_sql = f"""
    MERGE INTO {dbutils.widgets.get("catalog")}.silver.customers_scd2 AS target
    USING dfinsert1 AS source
    ON target.customer_id = source.customer_id AND target.is_current = TRUE AND target.customer_key = source.customer_key
    WHEN NOT MATCHED THEN
      INSERT (customer_key, customer_id, name, email, region, hash_value, start_date, end_date, is_current, _ingest_timestamp, _source_file_name)
      VALUES (source.customer_key, source.customer_id, source.name, source.email, source.region, source.hash_value, source.start_date, source.end_date, source.is_current, source._ingest_timestamp, source._source_file_name)
"""

spark.sql(merge_sql)

In [ ]:
%skip 
audit_log(spark, table_name, log_path)

In [ ]:
merge_sql = f"""
    MERGE INTO {dbutils.widgets.get("catalog")}.silver.customers_scd2 AS target
    USING (select customer_key, customer_id, name, email, region, hash_value, start_date, end_date, is_current, _ingest_timestamp, _source_file_name 
            from dfinsert1 
           union 
           select customer_key, customer_id, name, email, region, hash_value, start_date, end_date, is_current, _ingest_timestamp, _source_file_name 
            from dfinsert) AS source
    ON target.customer_id = source.customer_id AND target.is_current = TRUE AND target.customer_key = source.customer_key
    WHEN MATCHED AND target.hash_value <> source.hash_value THEN
      UPDATE SET target.is_current = FALSE, target.end_date = source.start_date
    WHEN NOT MATCHED THEN
      INSERT (customer_key, customer_id, name, email, region, hash_value, start_date, end_date, is_current, _ingest_timestamp, _source_file_name)
      VALUES (source.customer_key, source.customer_id, source.name, source.email, source.region, source.hash_value, source.start_date, source.end_date, source.is_current, source._ingest_timestamp, source._source_file_name)
"""

spark.sql(merge_sql)

In [ ]:
audit_log(spark, table_name, log_path)